In [2]:
import pandas as pd
from sklearn.calibration import LabelEncoder
import numpy as np
# 1. Load the data
df = pd.read_csv('./megablunders.csv')

In [3]:
# look at class imbalances

import matplotlib.pyplot as plt

# get the count for each label
print(df["error"].value_counts())

df_filtered = df[df["error"] != "AGREEERROR"]

# Verify the class has been removed
print("\nClass distribution after dropping:")
print(df_filtered["error"].value_counts())

# Update your DataFrame
df = df_filtered


print(df["error"].value_counts())

error
PR            14
PAR           14
ROS           14
MM            14
DM            13
AGREE         13
CASE          13
FRAG          12
NONE          11
AGREEERROR     1
Name: count, dtype: int64

Class distribution after dropping:
error
PR       14
PAR      14
ROS      14
MM       14
DM       13
AGREE    13
CASE     13
FRAG     12
NONE     11
Name: count, dtype: int64
error
PR       14
PAR      14
ROS      14
MM       14
DM       13
AGREE    13
CASE     13
FRAG     12
NONE     11
Name: count, dtype: int64


In [47]:
# use lable encoder to encode the error
label_encoder = LabelEncoder()
df['error'] = label_encoder.fit_transform(df['error'])

In [48]:
#  vectorize the text 
from sentence_transformers import SentenceTransformer

st = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

df = df.reset_index(drop=True)

In [49]:
# Convert to list of strings before encoding
sentences = df["original_sentence"].tolist()
embeddings = st.encode(sentences)

In [50]:
df["error"].nunique()

9

In [51]:
df.head()

,original_sentence,error
0,By dropping a game to the pathetic Tampa Bay D...,2
1,"Although, if history is any indication, the te...",3
2,"The Sox almost never go down uneventfully, whi...",7
3,Because of the accumulated bad karma that hang...,0
4,The team not only has squandered huge leads bu...,6


In [52]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['error'], test_size=0.1, random_state=42, stratify=df["error"])

In [ ]:
# Download the necessary NLTK data first
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

# Check if the data was downloaded successfully
try:
    from nltk.corpus import wordnet
    print("WordNet loaded successfully")
except:
    print("WordNet loading failed")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/felixstuart/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/felixstuart/nltk_data...


True

In [63]:
# use nplaug to expand the dataset
import nlpaug.augmenter.word as naw

syn_aug = naw.SynonymAug(aug_src='wordnet')

augmented_sentences = []
augmented_labels = []

for sentence, label in zip(X_train, y_train):
    augmented = syn_aug.augment(sentence)
    augmented_sentences.append(augmented)
    augmented_labels.append(label)
    
X_train_extended = X_train + augmented_sentences
y_train_extended = y_train + augmented_labels


NameError: name 'wordnet' is not defined

In [57]:
# try xgboost
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Convert class weights to sample weights
# Create a dictionary mapping class labels to weights
class_weight_dict = dict(zip(unique_classes, weights))

# Map each sample to its appropriate weight
sample_weights = np.array([class_weight_dict[cls] for cls in y_train])

# Fit the XGBoost model with sample weights
xboost = xgb.XGBClassifier(n_estimators=500,
                           learning_rate=0.01,
                           random_state=42,
                           reg_alpha=0.7,
                           reg_lambda=1,
                           max_depth=3,
                           scale_pos_weight=sample_weights,
                           )


xboost.fit(X_train_extended, y_train_extended, sample_weight=sample_weights)

# Make predictions
y_pred = xboost.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Accuracy: {accuracy*100:.2f}%")

NameError: name 'X_train_extended' is not defined